# Using of Processor

## BGE-M3 model

In [14]:
from processor import Preprocessor
from FlagEmbedding import BGEM3FlagModel

In [15]:
preprocessor = Preprocessor(filepath="ai_doc.pdf")

In [16]:
text = preprocessor.read_text()

In [17]:
clean_text = preprocessor.clean_text(text=text)

In [18]:
chunks = preprocessor.split_into_chunks(
    text=clean_text, 
    max_chunk_size=600
)

In [19]:
preprocessor.save_chunks(
    chunks=chunks,
    path="/home/murad/Documents/contextual_embeddings/chunks"
)

Error saving chunk to /home/murad/Documents/contextual_embeddings/chunks/chunk_0.txt: [Errno 2] No such file or directory: '/home/murad/Documents/contextual_embeddings/chunks/chunk_0.txt'
Error saving chunk to /home/murad/Documents/contextual_embeddings/chunks/chunk_1.txt: [Errno 2] No such file or directory: '/home/murad/Documents/contextual_embeddings/chunks/chunk_1.txt'
Error saving chunk to /home/murad/Documents/contextual_embeddings/chunks/chunk_2.txt: [Errno 2] No such file or directory: '/home/murad/Documents/contextual_embeddings/chunks/chunk_2.txt'
Error saving chunk to /home/murad/Documents/contextual_embeddings/chunks/chunk_3.txt: [Errno 2] No such file or directory: '/home/murad/Documents/contextual_embeddings/chunks/chunk_3.txt'
Error saving chunk to /home/murad/Documents/contextual_embeddings/chunks/chunk_4.txt: [Errno 2] No such file or directory: '/home/murad/Documents/contextual_embeddings/chunks/chunk_4.txt'
Error saving chunk to /home/murad/Documents/contextual_embed

In [20]:
embedding_model = BGEM3FlagModel(model_name_or_path="BAAI/bge-m3",
                                 use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [21]:
chunks_embeddings = embedding_model.encode(
    sentences=chunks,
    batch_size=12,
    max_length=8192,
    )['dense_vecs']

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
collection = preprocessor.create_database(
    chunks=chunks,
    embeddings=chunks_embeddings
)

Using ChromaDB directory: /tmp/tmpt7k_a_4z
Successfully stored 37 chunks with embeddings.
Database location: /tmp/tmpt7k_a_4z


In [23]:
query = "Turinq testi nədir?"
query_embedding = embedding_model.encode(
    sentences=query,
    batch_size=12,
    max_length=1024
)["dense_vecs"]
query_embedding = query_embedding.tolist()

In [24]:
collection.query(
    query_embeddings=query_embedding,
    n_results=10,
    include=["distances", "metadatas", "documents"]
)

{'ids': [['chunk_16',
   'chunk_17',
   'chunk_15',
   'chunk_26',
   'chunk_29',
   'chunk_35',
   'chunk_18',
   'chunk_22',
   'chunk_21',
   'chunk_1']],
 'distances': [[0.6031309300372137,
   0.7156820159450029,
   0.9340197892554905,
   1.069768826620262,
   1.1289514818185007,
   1.1640132529745169,
   1.195128053477976,
   1.2231285490888089,
   1.2289387843506814,
   1.2372925567881798]],
 'metadatas': [[{'source': 'chunk_16.txt'},
   {'source': 'chunk_17.txt'},
   {'source': 'chunk_15.txt'},
   {'source': 'chunk_26.txt'},
   {'source': 'chunk_29.txt'},
   {'source': 'chunk_35.txt'},
   {'source': 'chunk_18.txt'},
   {'source': 'chunk_22.txt'},
   {'source': 'chunk_21.txt'},
   {'source': 'chunk_1.txt'}]],
 'embeddings': None,
 'documents': [['Turinq, bir kompüterin süni zəkaya sahib ola biləcəyini və hakimi çaşdıracaq qədər inandırıcı ola biləcəyini iddia edir. Hakim, insanla yoxsa kompüterlə danışdığını anlamayacaq. Bu testə Turinq testi deyilir.Turinq testinin məqsədi və əh

## XLM-RoBERTa model

In [1]:
from processor import Preprocessor
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch

In [2]:
preprocessor = Preprocessor("ai_doc.pdf")

In [3]:
text = preprocessor.read_text()

In [4]:
clean_text = preprocessor.clean_text(text=text)

In [5]:
chunks = preprocessor.split_into_chunks(text=clean_text,
                                        max_chunk_size=600)

In [6]:
preprocessor.save_chunks(chunks=chunks,
                         path="/home/murad/Documents/self-study/contextual_embeddings/chunks")

In [7]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [8]:
model.eval()
embeddings = []

with torch.no_grad():
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        chunk_embedding = outputs.last_hidden_state[:, 0, :]
        embeddings.append(chunk_embedding)

In [10]:
collection = preprocessor.create_database(chunks=chunks,
                                          embeddings=list(embeddings))

Using ChromaDB directory: /tmp/tmpp7h95gnp


ValueError: Expected each embedding in the embeddings to be a list, got [tensor([[ 5.4107e-02,  1.2325e-01,  6.0962e-02, -2.1180e-02,  6.4997e-02,
         -4.7802e-02,  6.9524e-02, -1.2908e-02,  1.1306e-01, -1.6254e-01,
          5.3130e-02,  1.1418e-01, -6.9541e-02,  2.3060e-02, -2.9229e-02,
          4.9910e-02, -7.5564e-02, -4.0046e-02,  5.3028e-02,  1.2382e-01,
          2.8636e-02,  3.7272e-02,  1.3937e-01,  9.1373e-02, -6.6710e-02,
          2.3078e-03, -5.8499e-02,  7.5244e-02,  9.3449e-02, -3.1604e-02,
          1.1798e-01,  1.5202e-02,  8.5324e-02,  1.4643e-01,  1.5490e-01,
          5.2005e-03, -9.9057e-02, -7.7261e-02,  6.8537e-02,  8.7139e-02,
          6.3850e-02,  1.5367e-01,  2.6838e-03, -5.4182e-02,  4.2034e-03,
         -9.7592e-02, -1.0049e-01, -9.7114e-02, -3.1875e-02, -1.4837e-01,
          7.5907e-02, -3.0728e-02,  6.6148e-02,  7.0614e-02, -1.4303e-01,
         -1.0798e-01,  1.2868e-01,  8.8846e-02, -2.7375e-04,  1.3283e-01,
         -7.3816e-02,  1.0050e-01,  9.4